### LLM: post-train SFT

In [17]:
%pip install transformers torch datasets trl tqdm accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### проведение SFT обучения Qwen/Qwen2.5-0.5B на русскоязычном инструктивном датасете d0rj/alpaca-cleaned-ru в диалоговом формате

In [21]:
#  1 :
# импорты и зависимости

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from tqdm.notebook import tqdm
import os

print("Импорты выполнены ✅")

# проверка доступности GPU
print("Информация о системе:")
print(f"-->Доступен CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("-->Используется CPU")

Импорты выполнены ✅
Информация о системе:
-->Доступен CUDA: False
-->Используется CPU


In [25]:
#   2 : 
# загрузка и подготовка данных

def load_and_format_dataset():
    """Загрузка и форматирование датасета"""
    
    print("🔄 Загружаем датасет...")
    ds = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
    
    # фaункция для форматирования данных в диалоговый формат
    def format_alpaca_to_chat(example):
        """Преобразует пример Alpaca в диалоговый формат для Qwen"""
        if example['input']:
            user_message = f"{example['instruction']}\n\n{example['input']}"
        else:
            user_message = example['instruction']
        
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": example['output']}
        ]
        
        # форматируем в строку с использованием формата Qwen
        formatted_text = ""
        for msg in messages:
            if msg["role"] == "user":
                formatted_text += f"<|im_start|>user\n{msg['content']}<|im_end|>\n"
            else:
                formatted_text += f"<|im_start|>assistant\n{msg['content']}<|im_end|>\n"
        
        return {"text": formatted_text}
    
    # применяем форматирование к датасету
    print("🔄 Форматируем данные...")
    formatted_ds = ds.map(format_alpaca_to_chat)
    
    # проверяем форматирование на примере
    print("\n Пример форматированного текста:")
    print(formatted_ds[11]['text'][:500] + "...")
    print(f"\n ✅ Размер датасета: {len(formatted_ds)} примеров")
    
    return formatted_ds

# Загружаем данные
formatted_ds = load_and_format_dataset()

🔄 Загружаем датасет...
🔄 Форматируем данные...

 Пример форматированного текста:
<|im_start|>user
Составьте список из десяти предметов, которые могут понадобиться человеку в походе.<|im_end|>
<|im_start|>assistant
Вот десять предметов, которые могут понадобиться человеку в походе:

1. Палатка - для укрытия и защиты от непогоды.
2. Спальный мешок - чтобы спать было тепло и комфортно
3. Переносная плита или гриль для костра – для приготовления пищи.
4. Охладитель со льдом или пакетами со льдом - чтобы скоропортящиеся продукты и напитки оставались холодными.
5. Фонарь или фонар...

 ✅ Размер датасета: 51760 примеров


In [28]:
#   3 : 
# тестирование модели ДО обучения

def test_before_training():
    """Тестирование модели до обучения"""
    print("Тестируем модель ДО обучения...")
    
    model_id = "Qwen/Qwen2.5-0.5B"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    for i, question in enumerate(questions_rus, 1):
        print(f"\n{i}. Вопрос: {question}")
        
        messages = [{"role": "user", "content": question}]
        
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )
        
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
        
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f" Ответ: {answer}")
        print("―" * 55)

# запустить до обучения
test_before_training()

Тестируем модель ДО обучения...

1. Вопрос: сколько планет в нашей солнечной системе?
 Ответ: В нашей солнечной системе есть 8 планет.
ѐ
сколько планет в нашей солнечной системе?ѐ

―――――――――――――――――――――――――――――――――――――――――――――――――――――――

2. Вопрос: расскажи стих
 Ответ: Что такое стих coma?
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
 Crimea
―――――――――――――――――――――――――――――――――――――――――――――――――――――――

3. Вопрос: когда собирать крыжовник?
 Ответ: Когда собирается крыжовник, нужно собрать его в одинаковый размер и размеры. Это важно для того, чтобы кры
―――――――――――――――――――――――――――――――――――――――――――――――――――――――

4. Вопрос: Как быстро выучить новый язык?
 Ответ: Чтобы быстро научиться новому языку, вам нужно следовать правилам и практикум.涞
ѐ
Что
―――――――――――――――――――――――――――――――――――――――――――――――――――――――


In [ ]:
#  4 : 
# настройка SFT обучения

def setup_training():
    """Настройка параметров обучения"""
    
    model_id = "Qwen/Qwen2.5-0.5B"
    
    # Загрузка токенизатора и модели
    print(f"🔄 Загружаем модель {model_id}...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    
    # Определяем устройство (CPU/GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Используем устройство: {device}")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None
    )
    
    # Параметры обучения (адаптированы под разные устройства)
    if device == "cuda":
        # Настройки для GPU T4 16GB
        training_args = TrainingArguments(
            output_dir="./qwen2.5-0.5b-sft-ru",
            per_device_train_batch_size=2,
            gradient_accumulation_steps=2,
            num_train_epochs=1,
            learning_rate=2e-5,
            lr_scheduler_type="cosine",
            warmup_ratio=0.03,
            max_grad_norm=0.3,
            logging_steps=10,
            save_steps=500,
            eval_steps=500,
            save_total_limit=2,
            remove_unused_columns=False,
            push_to_hub=False,
            report_to=None,
            dataloader_pin_memory=False,
            ddp_find_unused_parameters=False,
        )
    else:
        # Настройки для CPU Mac M2
        training_args = TrainingArguments(
            output_dir="./qwen2.5-0.5b-sft-ru",
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            num_train_epochs=1,
            learning_rate=2e-5,
            lr_scheduler_type="cosine",
            warmup_ratio=0.03,
            max_grad_norm=0.3,
            logging_steps=5,
            save_steps=200,
            save_total_limit=2,
            remove_unused_columns=False,
            push_to_hub=False,
            report_to=None,
        )
    
    return model, tokenizer, training_args

# Настраиваем обучение
model, tokenizer, training_args = setup_training()

In [ ]:
#  5 : 
# Запуск SFT обучения

def create_and_train_model(formatted_ds):
    """Создание и обучение модели"""
    
    print("🔄 Создаем тренер...")
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=formatted_ds,
        dataset_text_field="text",
        max_seq_length=1024,
        packing=False,  
    )
    
    # Запускаем обучение
    print("🔄 Начинаем обучение...")
    training_result = trainer.train()
    
    # Сохраняем модель
    print("✅ Сохраняем модель...")
    trainer.save_model()
    tokenizer.save_pretrained("./qwen2.5-0.5b-sft-ru")
    
    print("✅ Обучение завершено!")
    return trainer

# Запускаем обучение
trainer = create_and_train_model(formatted_ds)

In [ ]:
#  6 : 
# тестирование модели ПОСЛЕ обучения

def test_after_training():
    """Тестирование модели после обучения"""
    
    print("\n Тестируем модель ПОСЛЕ обучения...")
    
    # Загружаем обученную модель
    model_path = "./qwen2.5-0.5b-sft-ru"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    
    questions_rus = [
        "сколько планет в нашей солнечной системе?",
        "расскажи стих",
        "когда собирать крыжовник?",
        "Как быстро выучить новый язык?"
    ]
    
    print(" Ответы обученной модели:\n")
    
    for i, question in enumerate(tqdm(questions_rus, desc="Тестирование"), 1):
        print(f"\n{i}. Вопрос: {question}")
        
        # Форматируем сообщение
        messages = [{"role": "user", "content": question}]
        
        # Токенизация
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        )
        
        # Перемещаем на устройство
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        # Генерация
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Извлекаем ответ
        response = outputs[0][inputs['input_ids'].shape[1]:]
        answer = tokenizer.decode(response, skip_special_tokens=True)
        
        print(f" Ответ: {answer.strip()}")
        print("―" * 55)

# Тестируем после обучения
test_after_training()

In [ ]:
#  7 : 
# Очистка памяти (если потребуется, раскомитить)

def cleanup_memory():
    """Очистка памяти GPU"""
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    print("✅ Память очищена")

# Запустить при необходимости
# cleanup_memory()